In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
data=pd.read_excel('ITEM table (주문량, 평균날짜, 스타일na값).xlsx')
data.head()

,상품명,색깔,카테고리,RGB,패턴재질,모양,계절,평균평점,스타일
0,[1+1] 기획특가♥라벨르 베이직셔츠,베이지,셔츠/남방,"229, 223, 201",무지,카라,봄 여름 가을,0.0,심플베이직
1,[1+1] 기획특가♥라벨르 베이직셔츠,핑크,셔츠/남방,"255, 191, 202",무지,카라,봄 여름 가을,0.0,심플베이직
2,[1+1] 기획특가♥웰데이 베이직셔츠,그레이,셔츠/남방,"169, 169, 169",무지,카라,여름,0.0,심플베이직
3,[1+1] 기획특가♥웰데이 베이직셔츠,라벤더,셔츠/남방,"210, 170, 224",무지,카라,여름,0.0,심플베이직
4,[1+1] 기획특가♥웰데이 베이직셔츠,블랙,셔츠/남방,"0, 0, 0",무지,카라,여름,0.0,심플베이직


In [3]:
data.dtypes

상품명      object
색깔       object
카테고리     object
RGB      object
패턴재질     object
모양       object
계절       object
평균평점    float64
스타일      object
dtype: object

In [4]:
data['상품이름']=data['상품명']+'-'+data['색깔']
data.head()

,상품명,색깔,카테고리,RGB,패턴재질,모양,계절,평균평점,스타일,상품이름
0,[1+1] 기획특가♥라벨르 베이직셔츠,베이지,셔츠/남방,"229, 223, 201",무지,카라,봄 여름 가을,0.0,심플베이직,[1+1] 기획특가♥라벨르 베이직셔츠-베이지
1,[1+1] 기획특가♥라벨르 베이직셔츠,핑크,셔츠/남방,"255, 191, 202",무지,카라,봄 여름 가을,0.0,심플베이직,[1+1] 기획특가♥라벨르 베이직셔츠-핑크
2,[1+1] 기획특가♥웰데이 베이직셔츠,그레이,셔츠/남방,"169, 169, 169",무지,카라,여름,0.0,심플베이직,[1+1] 기획특가♥웰데이 베이직셔츠-그레이
3,[1+1] 기획특가♥웰데이 베이직셔츠,라벤더,셔츠/남방,"210, 170, 224",무지,카라,여름,0.0,심플베이직,[1+1] 기획특가♥웰데이 베이직셔츠-라벤더
4,[1+1] 기획특가♥웰데이 베이직셔츠,블랙,셔츠/남방,"0, 0, 0",무지,카라,여름,0.0,심플베이직,[1+1] 기획특가♥웰데이 베이직셔츠-블랙


In [7]:
data.columns

Index(['상품이름', '카테고리', 'RGB', '패턴재질', '모양', '계절', '평균평점'], dtype='object')

In [8]:
data=data[['상품이름','카테고리','RGB','패턴재질','모양','계절','평균평점']]
data.head()

,상품이름,카테고리,RGB,패턴재질,모양,계절,평균평점
0,[1+1] 기획특가♥라벨르 베이직셔츠-베이지,셔츠/남방,"229, 223, 201",무지,카라,봄 여름 가을,0.0
1,[1+1] 기획특가♥라벨르 베이직셔츠-핑크,셔츠/남방,"255, 191, 202",무지,카라,봄 여름 가을,0.0
2,[1+1] 기획특가♥웰데이 베이직셔츠-그레이,셔츠/남방,"169, 169, 169",무지,카라,여름,0.0
3,[1+1] 기획특가♥웰데이 베이직셔츠-라벤더,셔츠/남방,"210, 170, 224",무지,카라,여름,0.0
4,[1+1] 기획특가♥웰데이 베이직셔츠-블랙,셔츠/남방,"0, 0, 0",무지,카라,여름,0.0


In [9]:
count_vector = CountVectorizer(ngram_range=(1,1))

In [10]:
c_vector1=count_vector.fit_transform(data['계절'])
c_vector2=count_vector.fit_transform(data['카테고리'])
c_vector5=count_vector.fit_transform(data['상품이름'])
# c_vector3=count_vector.fit_transform(data['모양'])
c_vector4=count_vector.fit_transform(data['계절'])
# c_vector6=count_vector.fit_transform(data['RGB'])

In [11]:
data.dtypes

상품이름     object
카테고리     object
RGB      object
패턴재질     object
모양       object
계절       object
평균평점    float64
dtype: object

In [12]:
genre_c_sim1=cosine_similarity(c_vector1,c_vector1).argsort()[:,::-1]
genre_c_sim2=cosine_similarity(c_vector2,c_vector2).argsort()[:,::-1]

In [13]:
genre_c_sim1.shape, genre_c_sim2.shape

((3503, 3503), (3503, 3503))

In [14]:
def get_recommend_item_list1(df,상품이름,top=30):
    #특정 영화와 비슷한 영화를추천해야 하기에 '특정 영화' 정보를 추출
    target_item_index=df[df['상품이름']==상품이름].index.values
    
    #코사인 유사도 중 비슷한 코사인 유사도를 가진 정보 추출
    sim_index1=genre_c_sim1[target_item_index,:top].reshape(-1)
    sim_index2=genre_c_sim2[target_item_index,:top].reshape(-1)
    #본인 제외
    sim_index1=sim_index1[sim_index1 != target_item_index]
    sim_index2=sim_index2[sim_index2 != target_item_index]
    #data frame로 만들고 vote_count로 정렬한뒤 return
    result1=df.iloc[sim_index1].sort_values('평균평점',ascending=False)[:10]
    result2=df.iloc[sim_index2].sort_values('평균평점',ascending=False)[:10]

    return result1

In [15]:
get_recommend_item_list1(data,'[DAYONE LABEL]누브 보트넥롱원피스-블랙')

,상품이름,카테고리,RGB,패턴재질,모양,계절,평균평점
1225,로아인 플라워 롱원피스-아이보리,롱원피스,"254, 255, 239",플라워,퍼프,봄 가을,5.000000
1206,로민 플레어 롱스커트-크림,롱스커트,"242, 234, 210",무지,플레어,봄 가을,5.000000
1127,런스 골지 나시&랩티set-핑크,민소매/나시,"231, 191, 192",무지,나시,봄 가을,5.000000
1224,로아인 플라워 롱원피스-라이트퍼플,롱원피스,"216, 192, 223",플라워,퍼프,봄 가을,5.000000
1168,레이지 셔링 버튼블라우스-아이보리,블라우스,"254, 255, 239",무지,셔링,봄 가을,5.000000
1152,레스티 슬릿 일자팬츠-블루,일자바지,"128, 163, 207",데님,일자,봄 가을,4.909091
1172,레일라 퍼프 롱원피스-핑크,롱원피스,"255, 191, 202",골지,퍼프,봄 가을,4.894737
1170,레일라 퍼프 롱원피스-그레이,롱원피스,"169, 169, 169",골지,퍼프,봄 가을,4.894737
1171,레일라 퍼프 롱원피스-아이보리,롱원피스,"254, 255, 239",골지,퍼프,봄 가을,4.894737
2951,커맨드 브이 골지가디건-블랙,가디건,"0, 0, 0",골지,브이넥,봄 가을,4.842105
